In [2]:
import torch
import numpy as np
import torch.nn.functional as F

# Graph Class Definition

It should consist of:

- `nodes`: All current node in the existing tree with their embedding (V set)
- `node_types`: all nodes in the existing tree with either leaf type or internal type
- `edge_source`: the start node of all existing edges
- `edge_dest`: the end node of all existing edges
- `leaf`: the remaining species the model could choose from for leaf nodes

**No branch length implemented for now!**

In [11]:
class Graph:
    def __init__(self, batch_size, embed_size, num_taxon, device):
        if batch_size is None:
            return
        self.batch_size = batch_size
        self.device = device
        # init all components in the graph
        # existing nodes in the graph: each row contains the embedding vector of the original sequence for 1 node
        self.nodes = torch.zeros(0, embed_size, dtype=torch.float32, device=device)
        # type of the existing nodes: 1 dim array, each index is a type of the node, either internal nodes or leaf nodes
        self.node_types = torch.zeros(0, dtype=torch.uint8, device=device)
        # starting point of each node:1 dim array, number indicate the origin node of edge i
        self.edge_source = torch.zeros(0, dtype=torch.long, device=device)
        # ending point of each node:1 dim array, number indicate the destination node of edge i
        self.edge_dest = torch.zeros(0, dtype=torch.long, device=device)
        # all remaining species to choose from
        self.leaf = torch.zeros(num_taxon, embed_size, device=device, dtype=torch.uint8)
        
        # FIXME
        # No edge feature for now...
        # not sure what it is for now, could be the graph selected for the current batch, will update
        self.owner_masks = torch.zeros(batch_size, 0, dtype=torch.uint8, device=device)
        self.last_inserted_node = torch.zeros(batch_size, dtype=torch.long, device=device)
        # current running graph in the batch
        self.running = torch.ones(batch_size, device=device, dtype=torch.uint8)


# Model Definition

## Propagator:
The message passing part of the model, we update the node vectors based on the current existing graph

If no node or no edges existed in the tree, the following model will not run and directly return the current graph feature matrix

For the layers of this part of the model:

1. `message_node`: get the node message from the vector embedding (embed_size -> message_size)
   - The message that comes out of this layer will be a vector of message_size for each existing nodes
   - The message that feeds into the next layer will be the addition of source and destination nodes from these vectors
3. `message_layer`: addition layer for `message_node` (message_size -> message_size)
4. `node_update_fn`: The Gated Recurrent Unit cells that pass the message along neighboring nodes (message_size -> embed_size)

The return matrix is the updated feature matrix of the existing tree.

**Unsure if how exactly `_reset_parameters` works, will investigate once the model is runnable**

In [12]:
class Propagator(torch.nn.Module):
    def __init__(self, embed_size, dropout):
        super().__init__()
        # The message size in the message-passing
        self.message_size = embed_size * 2
        # update all node vectors back to original embed size
        self.node_update_fn = torch.nn.GRUCell(self.message_size, embed_size)

        # Get the node message through a linear layer
        self.message_node = torch.nn.Linear(embed_size, self.message_size, bias=False)
        # second layer of message passing
        self.message_layer = torch.nn.Sequential(
            torch.nn.Tanh(),
            torch.nn.Linear(self.message_size, self.message_size)
        )
        self.dropout = torch.nn.Dropout(dropout)
        self._reset_parameters(embed_size)

    @staticmethod
    def _node_update_mask(graph: Graph, mask_override: torch.ByteTensor):
        return graph.owner_masks[graph.running if mask_override is None else mask_override].sum(0)>0

    def forward(self, graph: Graph, mask_override: torch.ByteTensor = None):
        # no node or edge in the graph, no need for message passing
        if graph.nodes.shape[0]==0 or graph.edge_source.shape[0]==0:
            return graph
        # get all node features from embedding
        node_features = self.message_node(graph.node)
        # get the source and destestion node features
        e1 = node_features.index_select(dim=0, index=graph.edge_source)
        e2 = node_features.index_select(dim=0, index=graph.edge_dest)
        messages = e1 + e2
        messages = self.message_layer(messages)
        messages = self.dropout(dropout)

        # concatnate the messages for all nodes
        # now the matrix contains a vector for each existing nodes, a result of addition of its destination and source nodes
        inputs = torch.zeros(graph.nodes.shape[0], self.message_size, device=graph.nodes.device,
                             dtype=graph.nodes.dtype).index_add_(0, graph.edge_dest, messages).\
                             index_add_(0, graph.edge_source, messages)

        inputs = self.dropout(inputs)
        
        # now we do message passing
        updated_nodes = self.node_update_fn(inputs, graph.nodes)
        # put the updated node into the graph set
        # only update the masked node, keep the unmasked one as original
        graph.nodes = torch.where(self._node_update_mask(graph, mask_override).unsqueeze(-1), updated_nodes, graph.nodes)

        return graph

    def _reset_parameters(self, embed_size):
        msg_gain = torch.nn.init.calculate_gain("tanh")
        #FIXME: not sure if it should be embed_size*3 or embed_size*2
        xavier_init(self.message_node, msg_gain, embed_size * 3, self.message_size)
        xavier_init(self.message_layer[1], 1)

        self.node_update_fn.bias_hh.data.fill_(0)
        self.node_update_fn.bias_ih.data.fill_(0)
        self.node_update_fn.bias_hh[:embed_size].data.fill_(1)


## Multilayer Propagator:
This class simply run propagator multiple times, the number of iteration is decided by me

In [13]:
class MultilayerPropagator(torch.nn.Module):
    def __init__(self, embed_size, n_iter, dropout):
        super().__init__()
        ## run propagators for n_iter times
        self.propagators = torch.nn.ModuleList([Propagator(embed_size, dropout) for i in range(n_iter)])

    def forward(self, graph: Graph, *args, **kwargs):
        for p in self.propagators:
            graph = p(graph, *args, **kwargs)
        return graph

## Aggregator:
This part of the model get the propagated node vectors and aggregate them into 1 single vector of higher dimension

The output of this model will be used as inputs for all the decision making NN in the later part of the bigger model

For the layers of this model:

1. `aggregate`: map the embedded(propagated) node vectors to a higher dimension (embed_size -> aggregated_size)
2. `gated_sum`: gating vector for the gated sum (embed_size -> aggregated_size)

What comes out of the above 2 NN is 2 `num_node * aggregated_size` matrices $g_{v}$ and $h_{v}$. We do dot product first, and add all rows of the dot product together, ie $\sum_{v\in V} g_{v} \odot h_{v}$ 

In the end, it becomes one `1*aggregated_size` vector for each tree.

In [14]:
class Aggregator(torch.nn.Module):
    def __init__(self, embed_size, dropout, bias_if_empty=False):
        super.__init__()

        self.aggregated_size = embed_size * 2
        # map embedding to a higher dimension
        self.aggregate = torch.nn.Linear(embed_size, aggregated_size)
        # part of the gated sum
        self.gated_sum = torch.nn.Sequential(
            torch.nn.Linear(embed_size, aggregated_size),
            torch.nn.Sigmoid()
        )
  
        self.bias_if_empty = torch.nn.Parameter(torch.Tensor(1, aggregated_size)) if bias_if_empty else None
        self.dropout = torch.nn.Dropout(dropout)

        self._reset_parameters()

    def forward(self, graph: Graph):
        # if no nodes exists in the current tree, return a vector full of zeros or the bias
        if graph.nodes.shape[0] == 0:
            if self.bias_if_empty is not None:
                return self.bias_if_empty.expand(graph.batch_size, -1)
            else:
                return torch.zeros(graph.batch_size, self.aggregated_size, dtype=torch.float32, device=graph.device)

        
        gates = self.gated_sum(graph.nodes)
        feature = self.aggregate(graph.nodes)

        # get the gated sum from the 2 NN
        fmask = graph.owner_masks.float()
        gated_sum = torch.mm(fmask, feature * gates)

        return self.dropout(gated_sum)

    def _reset_parameters(self):
        xavier_init(self.transform, 1)
        xavier_init(self.gate[0], 1)
        self.gate[0].bias.data.fill_(1)
        if self.bias_if_empty is not None:
            torch.nn.init.normal_(self.bias_if_empty)

## Add_Node

In [ ]:
class Add_Node(torch.nn.Module):
    def __init__(self, embed_size, aggregated_size, propagate_steps, dropout):
        super().__init__()
        self.propagator = MultilayerPropagator(embed_size, propagate_steps, dropout)
        self.decision_aggregator = Aggregator(embed_size, aggregated_size, dropout, bias_if_empty=True)
        self.generate_aggregator = Aggregator(embed_size, aggregated_size, dropout, bias_if_empty=True)

        # Decide whether to:
        # - Not add any node: 0
        # - Add an internal node: 1
        # - Add a leaf node: 2
        self.node_decision = torch.nn.Linear(aggregated_size, 3)
        # get the parameter of the node embedding
        self.node_embedding = torch.nn.Parameter(torch.Tensor(embed_size))
        # NN for generating the internal nodes
        self.init_1 = torch.nn.Linear(embed_size, embed_size)
        self.init_2 = torch.nn.Linear(aggregated_size, embed_size, bias=False)

        self._reset_parameters(embed_size, aggregated_size)

    def forward(self, graph:Graph):
        loss = 0
        # do a message passing before start the decision
        graph = self.propagator(graph)
        # make decision: add node or not? if so, what type?
        new_node_type = self.node_decision(self.decision_aggregator(graph))
        # Force model to add node if existing tree is empty
        if graph.node.shape[0] == 0:
            new_node_type[:, 0] = float("-inf")

        selected_node_type = sample_softmax(new_node_type)
        # if selected type is 0, terminate the whole algorithm
        graph.running = (selected_node_type != 0) & graph.running
        if graph.running.any():
            # Leaf node
            if selected_node_type == 1:
                #do leaf node thingy
            else:
                # internal nodes
                new_embedding = self.node_embedding
                # get the vector representing the whole graph 
                init_feature = self.generate_aggregator(graph)
                # We generate the feature for this internal node based on the existing
                new_feature = self.init_1(new_embedding) + self.init_2(init_feature)

                # Now we have the node feature for the new node, we add it
                mask = graph.running
                index_seq = torch.arange(mask.long().sum(), device = graph.device, dtype = torch.long) + \
                        (graph.nodes.shape[0] if graph.nodes is not None else 0)
                last_nodes = torch.zeros(graph.batch_size, device = graph.device, dtype = torch.long)
                last_nodes[mask] = index_seq

                # Select last node if updated
                graph.last_inserted_node = torch.where(mask, last_nodes, graph.last_inserted_node)
                # Select the new generated node features 
                new_node = new_feature[mask]
                # So here is how this line of code works...
                # mask is all the graph in the batch that is currently running, eg:[0,0,1,1,1,0,0,0,0,0]
                # mask.nonzero() gives the index of all the nonzero values, eg:[[2], [3], [4]]
                # mask.nonzero().squeeze(-1) remove the last dimension, eg: [2,3,4]
                # With the one_hot and batch_size classes, gives a one_hot matrix, eg: [0,0,1,0,0,0,0,0,0,0]
                #                                                                      [0,0,0,1,0,0,0,0,0,0]
                #                                                                      [0,0,0,0,1,0,0,0,0,0]
                # transpose() is just the transpose of this matrix
                owner_mask = F.one_hot(mask.nonzero().squeeze(-1), graph.batch_size).transpose(0,1).byte()
                graph.nodes = torch.cat((graph.nodes, new_nodes), dim=0)
                graph.owner_masks = torch.cat((graph.owner_masks, owner_masks), dim=1)
                
        return graph

In [5]:
def xavier_init(layer, scale, n_inputs=None, n_outputs=None):
    n_inputs = n_inputs if n_inputs is not None else layer.weight.shape[1]
    n_outputs = n_outputs if n_outputs is not None else layer.weight.shape[0]
    limits = scale * math.sqrt(6.0 / (n_inputs + n_outputs))
    layer.weight.data.uniform_(-limits, limits)

    if layer.bias is not None:
        torch.nn.init.normal_(layer.bias)

In [18]:
def sample_softmax(tensor, dim=-1):
    eps=1e-20

    # Built in gumbel softmax could end up with lots of nans. Do it manually here.
    noise = -torch.log(-torch.log(torch.rand_like(tensor)+eps) + eps)
    res = F.softmax(tensor + noise, dim=-1)
    _, res = res.max(dim=dim)
    return res

In [46]:
a = torch.zeros(10, dtype=torch.uint8)
a[2] = 1
a[3] = 1

x = F.one_hot(a.nonzero().squeeze(-1), 10).transpose(0,1).byte()
y = F.one_hot(a.nonzero().squeeze(-1), 10).transpose(0,1).byte()
torch.cat((x, y), dim=1)


tensor([[0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 0, 1, 0],
        [0, 1, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]], dtype=torch.uint8)